In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from six import string_types, iteritems


In [2]:
import os

In [3]:
import numpy as np
import tensorflow as tf


/opt/conda/envs/tfjs/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
import numpy as np


In [5]:
import detect_face

In [6]:
gpu_memory_fraction = 0.25

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
pnet, rnet, onet = detect_face.create_mtcnn(sess, None)


In [7]:
save_dir = './save'

In [8]:
!mkdir -p {save_dir}

In [9]:
saver = tf.train.Saver()

In [10]:
saver.save(sess, './save/tf_mtcnn', global_step=0)
saver.export_meta_graph('./save/tf_mtcnn_graph');
writer = tf.summary.FileWriter('./save')
writer.add_graph(tf.get_default_graph())
writer.flush()

see https://kpzhang93.github.io/MTCNN_face_detection_alignment/paper/spl.pdf for MTCNN P-/R-/O-net structure

see https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/README.md for SaveModel usage


see https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/protobuf/meta_graph.proto for SignatureDef


```
message SignatureDef {
  // Named input parameters.
  map<string, TensorInfo> inputs = 1;
  // Named output parameters.
  map<string, TensorInfo> outputs = 2;
  // Extensible method_name information enabling third-party users to mark a
  // SignatureDef as supporting a particular method. This enables producers and
  // consumers of SignatureDefs, e.g. a model definition library and a serving
  // library to have a clear hand-off regarding the semantics of a computation.
  //
  // Note that multiple SignatureDefs in a single MetaGraphDef may have the same
  // method_name. This is commonly used to support multi-headed computation,
  // where a single graph computation may return multiple results.
  string method_name = 3;
}
```

In [11]:
export_dir='./saved_model'

In [12]:
!rm -fr ./saved_model

In [13]:
builder = tf.saved_model.builder.SavedModelBuilder(export_dir)

In [14]:
g = tf.get_default_graph()

In [15]:
pnet_input_ = g.get_tensor_by_name('pnet/input:0')
pnet_output_1_ = g.get_tensor_by_name('pnet/conv4-2/BiasAdd:0')
pnet_output_2_ = g.get_tensor_by_name('pnet/prob1:0')
pnet_input = tf.saved_model.utils.build_tensor_info(pnet_input_)
pnet_output_1 = tf.saved_model.utils.build_tensor_info(pnet_output_1_)
pnet_output_2 = tf.saved_model.utils.build_tensor_info(pnet_output_2_)

In [16]:
pnet_signature = tf.saved_model.signature_def_utils.build_signature_def(
    inputs={
        tf.saved_model.signature_constants.CLASSIFY_INPUTS: pnet_input
    },
    outputs={
        tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES: pnet_output_1,
        tf.saved_model.signature_constants.CLASSIFY_OUTPUT_SCORES: pnet_output_2
    },
    method_name='mtcnn_pnet') # tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME)

In [17]:
rnet_input_ = g.get_tensor_by_name('rnet/input:0')
rnet_output_1_ = g.get_tensor_by_name('rnet/conv5-2/conv5-2:0')
rnet_output_2_ = g.get_tensor_by_name('rnet/prob1:0')
rnet_input = tf.saved_model.utils.build_tensor_info(rnet_input_)
rnet_output_1 = tf.saved_model.utils.build_tensor_info(rnet_output_1_)
rnet_output_2 = tf.saved_model.utils.build_tensor_info(rnet_output_2_)

In [18]:
rnet_signature = tf.saved_model.signature_def_utils.build_signature_def(
    inputs={
        tf.saved_model.signature_constants.CLASSIFY_INPUTS: rnet_input
    },
    outputs={
        tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES: rnet_output_1,
        tf.saved_model.signature_constants.CLASSIFY_OUTPUT_SCORES: rnet_output_2
    },
    method_name='mtcnn_rnet') # tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME)

In [19]:
onet_input_ = g.get_tensor_by_name('onet/input:0')
onet_output_1_ = g.get_tensor_by_name('onet/conv6-2/conv6-2:0')
onet_output_2_ = g.get_tensor_by_name('onet/conv6-3/conv6-3:0')
onet_output_3_ = g.get_tensor_by_name('onet/prob1:0')
onet_input = tf.saved_model.utils.build_tensor_info(onet_input_)
onet_output_1 = tf.saved_model.utils.build_tensor_info(onet_output_1_)
onet_output_2 = tf.saved_model.utils.build_tensor_info(onet_output_2_)
onet_output_3 = tf.saved_model.utils.build_tensor_info(onet_output_3_)

In [20]:
onet_1_signature = tf.saved_model.signature_def_utils.build_signature_def(
    inputs={
        tf.saved_model.signature_constants.CLASSIFY_INPUTS: onet_input
    },
    outputs={
        tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES: onet_output_1,
        tf.saved_model.signature_constants.CLASSIFY_OUTPUT_SCORES: onet_output_3
    },
    method_name='mtcnn_onet_1') # tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME)

In [21]:
onet_2_signature = tf.saved_model.signature_def_utils.build_signature_def(
    inputs={
        tf.saved_model.signature_constants.CLASSIFY_INPUTS: onet_input
    },
    outputs={
        tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES: onet_output_2,
        tf.saved_model.signature_constants.CLASSIFY_OUTPUT_SCORES: onet_output_3
    },
    method_name='mtcnn_onet_2') # tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME)

In [22]:
?builder.add_meta_graph_and_variables

Signature: builder.add_meta_graph_and_variables(sess, tags, signature_def_map=None, assets_collection=None, legacy_init_op=None, clear_devices=False, main_op=None, strip_default_attrs=False)
Docstring:
Adds the current meta graph to the SavedModel and saves variables.

Creates a Saver to save the variables from the provided session. Exports the
corresponding meta graph def. This function assumes that the variables to be
saved have been initialized. For a given `SavedModelBuilder`, this API must
be called exactly once and for the first meta graph to save. For subsequent
meta graph defs to be added, the `add_meta_graph()` API must be used.

Args:
  sess: The TensorFlow session from which to save the meta graph and
    variables.
  tags: The set of tags with which to save the meta graph.
  signature_def_map: The map of signature def map to add to the meta graph
    def.
  assets_collection: Assets collection to be saved with SavedModel.
  legacy_init_op: Legacy support for op or group of 

In [23]:
builder.add_meta_graph_and_variables(sess,[tf.saved_model.tag_constants.SERVING],
                                    signature_def_map = {
                                        "mtcnn_pnet": pnet_signature,
                                        "mtcnn_rnet": rnet_signature,
                                        "mtcnn_onet_1": onet_1_signature,
                                        "mtcnn_onet_2": onet_2_signature,
                                    })
builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_model/saved_model.pb


'./saved_model/saved_model.pb'

In [26]:
%%bash
export PATH=$CONDA_PREFIX/bin:$PATH
mkdir -p web
tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_node_names='pnet/conv4-2/BiasAdd,pnet/prob1,rnet/conv5-2/conv5-2,rnet/prob1,onet/conv6-2/conv6-2,onet/conv6-3/conv6-3,onet/prob1' \
    --saved_model_tags=serve \
    ./saved_model \
    ./web/web_model

Converted 50 variables to const ops.
Writing weight file ./web_model/tensorflowjs_model.pb...


/opt/conda/envs/tfjs/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
Instructions for updating:
Use the retry module or similar alternatives.
2018-05-24 10:31:03.385878: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2018-05-24 10:31:03.521037: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-05-24 10:31:03.521350: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1344] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRa

In [27]:
!ls -lR web_model

web_model:
total 1960
-rw-rw-r-- 1 rhee rhee 1983424  5월 24 10:31 group1-shard1of1
-rw-rw-r-- 1 rhee rhee   15236  5월 24 10:31 tensorflowjs_model.pb
-rw-rw-r-- 1 rhee rhee    3825  5월 24 10:31 weights_manifest.json


In [ ]:
assert 0

In [ ]:
!tensorboard --logdir save --port 6006

/opt/conda/envs/tfjs/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-05-24 10:31:57.631646: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2018-05-24 10:31:57.748339: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-05-24 10:31:57.748635: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1344] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.645
pciBusID: 0000:01:00.0
totalMemory: 10.91GiB freeMemory: 6.74GiB
2018-05-24 10:31:57